[Microsoft System Numerics](https://docs.microsoft.com/de-de/dotnet/api/system.numerics?view=net-6.0)

In [ ]:
using System.Numerics;

public class Compute2D
{       
    /// <summary>
    /// Compute SpiralPoints in XY coordinate system, starting from centerPoint, with stepSize and stepRange
    /// </summary>
    /// <param name="centerPoint"></param>
    /// <param name="stepSize"></param>
    /// <param name="stepRange"></param>
    /// <returns>collection of Vectors</returns>
    public IEnumerable<Vector2> SpiralPoints(Vector2 centerPoint, float stepSize, float stepRange)
    {
        // sanity check
        if (stepSize > stepRange || stepSize < 0f || stepRange <0f)
            return new List<Vector2>() { centerPoint };

        int size = (int)(stepRange / stepSize);
        var series = new List<Vector2>();            
        var center = new Vector2(size/2, size/2);
        var points = SpiralPoints(size);
                    
        foreach(var point in points)
        {                
            var shift = Vector2.Subtract(point, center);                
            var scale = new Vector2(shift.X * stepSize, shift.Y * stepSize);                
            series.Add(Vector2.Add(centerPoint, scale));
        }

        return series;
    }

    /// <summary>
    /// SpiralPoints
    /// </summary>
    /// <param name="size"></param>
    /// <returns></returns>
    private IEnumerable<Vector2> SpiralPoints(int size)
    {
        Vector2 point = new Vector2(size / 2, size / 2);

        yield return point;
        int sign = 1;
        for (int row = 1; row < size; row++)
        {
            // move right/left by row, and then up/down by row
            for (int k = 0; k < row; k++)
            {
                point = Vector2.Add(point, new Vector2(sign, 0));                    
                yield return point;
            }
            for (int k = 0; k < row; k++)
            {                    
                point = Vector2.Add(point, new Vector2(0, -sign));
                yield return point;
            }
            sign *= -1;
        }
        // last leg to finish filling the area
        for (int k = 0; k < size - 1; k++)
        {                
            point = Vector2.Add(point, new Vector2(sign, 0));
            yield return point;
        }
    }

    /// <summary>
    /// Compute ping pong points in XY coordinate system, starting from centerPoint, with stepSize and stepRange
    /// </summary>
    /// <param name="centerPoint"></param>
    /// <param name="stepSize"></param>
    /// <param name="stepRange"></param>
    /// <returns>collection of Vectors</returns>
    public IEnumerable<Vector2> PingPongPoints(Vector2 centerPoint, float stepSize, float stepRange, bool isHorizontal = true)
    {
        // sanity check
        if (stepSize > stepRange || stepSize < 0f || stepRange < 0f)
            return new List<Vector2>() { centerPoint };

        int size = (int)(stepRange / stepSize);
        var series = new List<Vector2>();
        var points = PingPongPoints(size);
        
        float offset = 0f; // test for visualizing the points in a plot
        foreach (var point in points)
        {
            var shift = isHorizontal ? new Vector2(point, offset) : new Vector2(offset, point);
            var scale = new Vector2(shift.X * stepSize, shift.Y * stepSize);
            series.Add(Vector2.Add(centerPoint, scale));
            //offset+=0.1f;
        }
        return series;
    }

    /// <summary>
    /// PingPongPoints
    /// </summary>
    /// <param name="size">collection of vectors</param>
    private IEnumerable<int> PingPongPoints(int size)
    {
        yield return 0;
        for (int coordinate = 1; coordinate <= size; coordinate++)
        {
            // add a point in the current direction
            yield return coordinate;
            yield return -coordinate;
        }
    }
}

[plotly](https://plotly.net/)

In [ ]:
// c# plot the points using xplot

#r "nuget:XPlot.Plotly"

In [ ]:
using XPlot.Plotly;

public class Plot
{
    public static void ScatterPlot(IEnumerable<float> x, IEnumerable<float> y, string title = "Scatter Plot", string xTitle = "X", string yTitle = "Y")
    {
        var chart = Chart.Plot(
            new Scatter
            {
                x = x,
                y = y,
                mode = "lines+markers"
            }
        );

        var chart_layout = new Layout.Layout{
            title=title,
            xaxis =new Xaxis{
                title = xTitle
                },
            yaxis =new Yaxis{
            title = yTitle
                }           
            };

        chart.WithLayout(chart_layout);
        chart.Show();
    }
}


...and here is the sample to process..

In [ ]:
var test = new Compute2D();
//var samples = test.PingPongPoints(new Vector2(0, 0), 1f, 3f, true );
var samples = test.PingPongPoints(new Vector2(1.5f, 0.5f), 0.8f, 4.7f, true );
Console.WriteLine(string.Join(",", samples));
Plot.ScatterPlot(samples.Select(x => x.X).ToList(), samples.Select(x => x.Y).ToList(), "Ping Pong Points", "X", "Y" );

In [ ]:
var compute = new Compute2D();
//var series = compute.SpiralPoints(5);            
//var series = compute.SpiralPoints(new Vector2(0, 0), 1f, 3f);
//var series = compute.SpiralPoints(new Vector2(-1.4f, -1.4f), 0.7f, 3.1f);
var series = compute.SpiralPoints(new Vector2(1f, 4f), 1.5f, 6.1f);

Plot.ScatterPlot(series.Select(x => x.X).ToList(), series.Select(x => x.Y).ToList(), "Spiral Points", "X", "Y" );